In [2]:
import torch

from loadDataset import subjectLoader
from preprocess import preprocess
from train import train_validate
import lossOption
# from test import test
from util import plot_loss_time
from util import prepare_batch, get_model_and_optimizer
from visualization_result import plot

PermissionError: [Errno 13] Permission denied: '/gpfs/data/yuanlab/projects/HN_autocontour/notebooks/3D_Unet_Dante/test.py'

In [2]:
organ = 'BrainStem'
mask_name = f'mask_{organ}'

#### prepare data

In [5]:
training_subjects, validation_subjects, test_subjects = subjectLoader().__getitem__()
training_set, validation_set, test_set = preprocess().transformation(training_subjects, validation_subjects, test_subjects)
print('Training set:', len(training_set), 'subjects')
print('Validation set:', len(validation_set), 'subjects')
print('Test set:', len(test_set), 'subjects')

Training set: 211 subjects
Validation set: 45 subjects
Test set: 46 subjects


#### train

In [5]:
save_to = f'team_models/script_testing.pth'
param = {'Name': [lossOption.Dice(), lossOption.BCE()], 'Weights': [0.5, 0.5]}
scheduled_param = {'First': [lossOption.BCE()],
                   'FirstWeights': [1],
                   'Second': [lossOption.Dice(), lossOption.BCE()],
                   'SecondWeights': [0.3, 0.7],
                   'Scheduler': [0.05, 0]}
obj = train_validate(organ=organ, 
                     save_to=save_to, 
                     num_epoch=1, 
                     loss_method='scheduled', 
                     scheduled_param=scheduled_param)

In [6]:
model, train_losses, val_losses = obj.run_train_val(training_set, validation_set)

Starting epoch 1


  0%|          | 0/211 [00:00<?, ?it/s]

/gpfs/data/yuanlab/projects/HN_autocontour/notebooks/3D_Unet_Dante/util.py:12: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  targets = (batch['mask']['data'] // 255).to(device)
/gpfs/share/apps/anaconda3/gpu/5.2.0/envs/yz7005/lib/python3.9/site-packages/unet/decoding.py:146: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  half_crop = cr

Training mean loss: 0.104


  0%|          | 0/45 [00:00<?, ?it/s]

Validation mean loss: 0.013


#### test

In [6]:
# if want to load from outside dictionary
state_dict = torch.load(f'team_models/{organ}_4dice_6boundary_state_dict.pth') 
loadmodel, _ = get_model_and_optimizer(torch.device('cuda'))
loadmodel.load_state_dict(state_dict['weights'])
train_losses = state_dict['train_losses']
val_losses = state_dict['val_losses']

In [7]:
test_obj = test(threshold='top_prob', organ=organ)
probability, target, dice, ppv, sensitivity = test_obj.run_test(test_set, loadmodel)

  0%|          | 0/46 [00:00<?, ?it/s]

/gpfs/data/yuanlab/projects/HN_autocontour/notebooks/3D_Unet_Dante/util.py:12: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  targets = (batch['mask']['data'] // 255).to(device)
/gpfs/share/apps/anaconda3/gpu/5.2.0/envs/yz7005/lib/python3.9/site-packages/unet/decoding.py:146: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  half_crop = cr

dice mean: 0.406
dice std: 0.046
ppv mean: 0.610
ppv std: 0.107
sensitivity mean: 0.308
sensitivity std: 0.038
